In [253]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import codecs
import re
import os
from unidecode import unidecode
from bs4 import UnicodeDammit
import chardet
import locale

In [166]:
def convert_unicode_chars(text):
    text_unicode = unidecode(text.replace('&#160;',' '))
    return text_unicode

In [165]:
def regex_preprocessing(text):
    new_text = text.strip().lower().replace('\n',' ').replace('\t',' ')
    return new_text

In [191]:
def fun(url):
    flag = -1
    content = urlopen(url)
    soup = BeautifulSoup(content,"lxml")   
    
    alltags = soup.find_all()
    text = convert_unicode_chars(soup.get_text())  #All formatting characters like \n and \t are written as is.
    
    bold_tags = False  
    
    for tag in alltags:

        try:
            tag_text = re.sub('\s+', ' ', tag.get_text().strip())
            if tag.name=='b' or tag.name =='strong' or 'bold' in tag['style']:

                if '2.01' in tag_text :
                    flag = 1                    
        except:
            pass
    
    if flag == 1:
        return (text,flag)
    else:   
        text,flag = notbold(soup)
        if flag == 1:
            return (text,flag)
        else:
            text,flag = no_tags(soup)
            if flag == 1:
                return (text,flag)
            else:
                print ('No item #############', url)
                #print (soup.get_text())
                return ('No item 2.01',flag)

def notbold(soup):
    alltags = soup.find_all()
    text = convert_unicode_chars(soup.get_text())
    flag = -1
    k = re.compile('.*item 2.01[\.]*[ ]*(completion)+|.*item 2.01[\.]*[ ]*(disposition)+')
    for tag in alltags:    
        tag_text = regex_preprocessing(tag.get_text())
        if k.match(tag_text):
            flag = 1
            return (text,flag)
    if flag == -1:
        text,flag = no_tags(soup)
        return (text,flag)

def no_tags(soup):
    text = convert_unicode_chars(soup.get_text())
    flag = -1
    k = re.compile('.*item 2.01[\.]*[ ]*(completion)+|.*item 2.01[\.]*[ ]*(disposition)+')
    compare_text = regex_preprocessing(text)
    if k.match(compare_text):
        flag = 1
        return (text,flag)
    if flag == -1:
        return ('No item 2.01',flag)

In [222]:
def getText(home_url,year,quarter):
    path = 'Item 2.01 filings/'+year+'/Q'+str(quarter)+'/'
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    content = urlopen(home_url).read()
    soup = BeautifulSoup(content)
    if 'No Results were Found' in soup.get_text():
        print('The end!')
        return 1
    
    for link in soup.find_all(id='viewFiling'):  
        
        if link.string[:3] == '8-K':            # Ensures that only 8-K filings, and no attachments are downloaded.
            pos_arr = []                        # To find the position of the url and the company name
            a = (link.get('href'))
            for i in range(len(a)):
                if a[i] == "'":
                    pos_arr.append(i)
            url = a[pos_arr[0]+1:pos_arr[1]]
            company_name = a[pos_arr[2]+1:pos_arr[3]]
            ans,flag = fun(url)
            if flag == 1:
                with codecs.open(path+company_name+".txt","w",encoding='utf-8') as f:
                        f.write(ans)                               


In [211]:
years = ['2015','2016','2017','2018']
quarters = [('01/01/','03/31/'),('04/01/','06/30/'),('07/01/','09/30/'),('10/01/','12/31/')]

In [220]:
for year in years:    
    for quarter in quarters:
        print ('Year : ',year,'Quarter : Q',quarters.index(quarter)+1)
        if year == '2015' and quarters.index(quarter) == 0:      # Records are available only from 03/20/2015
            continue
        end = 0
        page_num = 1
        while not(end):
            if page_num == 1:
                home_url = "https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=2.01&sort=Date&formType=Form8K&isAdv=true&stemming=false&numResults=100&fromDate="+quarter[0]+year+"&toDate="+quarter[1]+year+"&numResults=100"
                print('Page number = ',page_num)
                page_num += 1
            else:
                home_url = "https://searchwww.sec.gov/EDGARFSClient/jsp/EDGAR_MainAccess.jsp?search_text=2.01&sort=Date&startDoc="+str(((page_num-1)*100)+1)+"&numResults=100&isAdv=true&formType=Form8K&fromDate="+quarter[0]+year+"&toDate="+quarter[1]+year+"&stemming=false"
                print('Page number = ',page_num)
                page_num += 1
                
            end = getText(home_url,year,quarters.index(quarter)+1)

Year :  2015 Quarter : Q 1
Year :  2015 Quarter : Q 2
Page number =  1
Page number =  2
No item ############# http://www.sec.gov/Archives/edgar/data/844856/000149315215002644/form8k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1561032/000114420415038427/v413841_8k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1468516/000119312515231375/d946351d8k.htm
Page number =  3
No item ############# http://www.sec.gov/Archives/edgar/data/101778/000010177815000020/mro-form8kjune2015debtclos.htm
Page number =  4
No item ############# http://www.sec.gov/Archives/edgar/data/1584754/000161577415001492/s101325_form8k.htm
Page number =  5
Page number =  6
Page number =  7
No item ############# http://www.sec.gov/Archives/edgar/data/1507181/000155479515000248/myhi0526form8k.htm
Page number =  8
Page number =  9
No item ############# http://www.sec.gov/Archives/edgar/data/1507181/000155479515000186/myhi0511form8k.htm
Page number =  10
No item ############# http://www

Page number =  2
No item ############# http://www.sec.gov/Archives/edgar/data/1539680/000107878216002980/f8k061416_8k.htm
Page number =  3
No item ############# http://www.sec.gov/Archives/edgar/data/1144216/000114421616000007/MKBY_20215_8k.txt
Page number =  4
No item ############# http://www.sec.gov/Archives/edgar/data/1501112/000147793216010758/mixx_8k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1502152/000116169716000877/form_8-k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/875159/000087515916000143/a8-k2016gltsandopex.htm
Page number =  5
No item ############# http://www.sec.gov/Archives/edgar/data/908311/000110465916124863/a16-12197_18k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1508381/000114420416105731/v441320_8k.htm
Page number =  6
No item ############# http://www.sec.gov/Archives/edgar/data/1488419/000114420416104358/v440781_8k.htm
Page number =  7
No item ############# http://www.sec.gov/Archives/edgar/data/110

Page number =  3
Page number =  4
No item ############# http://www.sec.gov/Archives/edgar/data/1169770/000165495417005511/banc8k_june92017.htm
Page number =  5
No item ############# http://www.sec.gov/Archives/edgar/data/774569/000119312517186927/d405037d8k.htm
Page number =  6
No item ############# http://www.sec.gov/Archives/edgar/data/1477336/000119312517183887/d368480d8k.htm
Page number =  7
No item ############# http://www.sec.gov/Archives/edgar/data/922487/000114036117021028/form8k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1488419/000114420417027881/v467194_8k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1509745/000110465917032074/a17-11578_38k.htm
Page number =  8
No item ############# http://www.sec.gov/Archives/edgar/data/1509745/000110465917032074/a17-11578_38k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1553023/000155302317000095/sentinel8-k.htm
Page number =  9
No item ############# http://www.sec.gov/Archives/

Page number =  4
Page number =  5
Page number =  6
Page number =  7
No item ############# http://www.sec.gov/Archives/edgar/data/1004989/000143774918010318/sgrp20180517_8k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1314196/000121390018006362/f8k050918_theolbgroup.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1327978/000132797818000059/a051518form8-kaprilnav.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1585101/000158510118000037/a2018nav8-kforsx3offering.htm
Page number =  8
No item ############# http://www.sec.gov/Archives/edgar/data/1521847/000119312518156772/d583326d8k.htm
Page number =  9
No item ############# http://www.sec.gov/Archives/edgar/data/715957/000119312518151870/d540148d8k.htm
Page number =  10
No item ############# http://www.sec.gov/Archives/edgar/data/1286681/000119312518139051/d574317d8k.htm
No item ############# http://www.sec.gov/Archives/edgar/data/1108348/000119312518133750/d486952d8k.htm
Page number = 